In [4]:
from typing import (Tuple,List,Dict,Union,Callable)
import pandas as pd
import numpy as np

from scr.activation_funcs import identity, ReLU, sign, tanh
from scr.binary_operators import add, diff, get_x, get_y, multiple, x_is_greater_than_y
import matplotlib.pyplot as plt

In [11]:
# import torch
# from torch.nn import (Tanh,ReLU)

# def dynamics(y_t:torch.tensor,sigma):
    
#     a,b = y_t
#     y_next = torch.zeros_like(y_t)
#     size = y_t.shape[1]
#     relu = ReLU()

#     y_next[0] =  1.0 * torch.tanh(a) + 0.8 * a * b + 1.0 * b - 1.0 * relu(torch.minimum(a,b)) + sigma * torch.randn(size)
#     y_next[1] = 0.6 * torch.sign(b) + 0.5 * a * b - 1.0 * torch.maximum(a,b) + sigma * torch.randn(size)
    
#     return y_next

# sigma_true = 0.1

# T_total = 2000
# y = torch.zeros((2, T_total))
# y_without_noise = torch.zeros((2, T_total))

# name_stock = dynamics(y,sigma_true)
# noise = dynamics(name_stock,0)


# # Nomura = name_stock[0]
# # PFN = name_stock[1]
# name_stock[:,0] = torch.tensor([0.5,0.5])
# noise[:,0] = torch.tensor([0.5,-0.5])
# name_stock_label = ["Nomura", "PFN"]

# plt.figure(figsize=(18,5))
# plt.xlim([T_total-100,T_total])
# plt.plot(name_stock[0].numpy(), color = "#cc0000", label = name_stock_label[0])
# plt.plot(name_stock[1].numpy(), color = "#083090", label = name_stock_label[1])
# plt.plot(noise[0].numpy(), color = "#cc0000", linestyle = "--", label = name_stock_label[0] + "(w/o noise)")
# plt.plot(noise[1].numpy(), color = "#083090", linestyle = "--", label = name_stock_label[1] + "(w/o noise)")
# plt.xlabel("time", fontsize = 18)
# plt.ylabel("y", fontsize = 18)
# plt.legend()

In [1]:
class Trader():
    def __init__(self, num_stock: int, num_factors_max: int, delay_time_max: int, activation_funcs: List[Callable], binary_operators: List[Callable], time_window: int = None) -> None:
        # set hyperparameters
        self.num_stock: int = num_stock
        self.num_factors_max: int = num_factors_max
        self.delay_time_max: int = delay_time_max
        self.activation_funcs: List[Callable] = activation_funcs
        self.binary_operators: List[Callable] = binary_operators
        # if time_window==None, train by using all data
        self.time_window: int = time_window

        # GMM字典
        self.dict_activation: Dict = dict(
            zip(activation_funcs, range(len(activation_funcs))))
        self.dict_binary: Dict = dict(
            zip(binary_operators, range(len(binary_operators))))

        # initialize by stock
        # initialized by uniform distribution
        self.num_factors: np.ndarray = np.random.choice(num_factors_max)
        self.delay_P: np.ndarray = np.random.choice(delay_time_max, num_stock)
        self.delay_Q: np.ndarray = np.random.choice(delay_time_max, num_stock)

        self.stock_P: List[np.ndarray] = [np.random.choice(
            num_stock, i) for i in self.num_factors]
        self.stock_Q: List[np.ndarray] = [np.random.choice(
            num_stock, i) for i in self.num_factors]

        self.activation_func: List = [np.random.choice(
            activation_funcs, i) for i in self.num_factors]
        self.binary_operator: List = [np.random.choice(
            binary_operators, i) for i in self.num_factors]

        self.w: List[np.ndarray] = [
            np.random.randn(i) for i in self.num_factors]

        self.X_factors: List[np.ndarray] = [
            np.zeros((0, i)) for i in self.num_factors]
        self.cumulative_error: List[np.ndarray] = np.zeros(num_stock)

    def set_params(self, i_stock, list_params):
        self.num_factors[i_stock] = list_params[0]["num_factor"]

        self.delay_P[i_stock] = [list_params[factor+1]["delay_P"]
                                 for factor in range(self.num_factors[i_stock])]
        self.delay_Q[i_stock] = [list_params[factor+1]["delay_Q"]
                                 for factor in range(self.num_factors[i_stock])]
        self.stock_P[i_stock] = [list_params[factor+1]["stock_P"]
                                 for factor in range(self.num_factors[i_stock])]
        self.stock_Q[i_stock] = [list_params[factor+1]["stock_Q"]
                                 for factor in range(self.num_factors[i_stock])]
        self.activation_func[i_stock] = [self.activation_funcs[list_params[factor+1]
                                                               ["activation_func"]] for factor in range(self.num_factors[i_stock])]
        self.binary_operator[i_stock] = [self.binary_operators[list_params[factor+1]
                                                               ["binary_operator"]] for factor in range(self.num_factors[i_stock])]

        self.w[i_stock] = np.random.randn(self.num_factors[i_stock])

        self.X_factors[i_stock] = np.zeros((0, self.num_factors[i_stock]))
        self.cumulative_error[i_stock] = 0.0

    def get_params(self, i_stock):
        list_params = []
        num_factor = self.num_factors[i_stock]
        list_params.append({"num_factor": num_factor})

        for factor in range(num_factor):
            dict_ = {}
            dict_["delay_P"] = self.delay_P[i_stock][factor]
            dict_["delay_Q"] = self.delay_Q[i_stock][factor]
            dict_["stock_P"] = self.stock_P[i_stock][factor]
            dict_["stock_Q"] = self.stock_Q[i_stock][factor]
            dict_[
                "activation_func"] = self.dict_activation[self.activation_func[i_stock][factor]]
            dict_[
                "binary_operator"] = self.dict_binary[self.binary_operator[i_stock][factor]]

            list_params.append(dict_)

        return list_params

    def reset_params(self, i_stock):
        # initialized by uniform distribution
        self.num_factors[i_stock] = np.random.choice(
            range(1, self.num_factors_max+1))
        self.delay_P[i_stock] = np.random.choice(
            self.delay_time_max, self.num_factors[i_stock])
        self.delay_Q[i_stock] = np.random.choice(
            self.delay_time_max, self.num_factors[i_stock])
        self.stock_P[i_stock] = np.random.choice(
            self.num_stock, self.num_factors[i_stock])
        self.stock_Q[i_stock] = np.random.choice(
            self.num_stock, self.num_factors[i_stock])
        self.activation_func[i_stock] = np.random.choice(
            self.activation_funcs, self.num_factors[i_stock])
        self.binary_operator[i_stock] = np.random.choice(
            self.binary_operators, self.num_factors[i_stock])
        self.w[i_stock] = np.random.randn(self.num_factors[i_stock])

        self.X_factors[i_stock] = np.zeros((0, self.num_factors[i_stock]))
        self.cumulative_error[i_stock] = 0.0

    def calc_factor(self, data, i_stock, j):
        """ i_stock番目の株に関してj番目の項を計算
        """
        Aj = self.activation_func[i_stock][j]
        Oj = self.binary_operator[i_stock][j]
        Pj = self.stock_P[i_stock][j]
        Qj = self.stock_Q[i_stock][j]
        Dj = self.delay_P[i_stock][j]
        Fj = self.delay_Q[i_stock][j]
        return Aj(Oj(data[Pj][self.delay_time_max-Dj], data[Qj][self.delay_time_max-Fj]))

    def calc_factors(self, data, i_stock):
        num_factors = self.num_factors[i_stock]
        factors = np.zeros(num_factors)
        for j in range(num_factors):  # 各ファクターごとに
            factors[j] = self.calc_factor(data, i_stock, j)
        return factors

    def stack_factors(self, data, i_stock):
        """
        time_windowの数だけ計算した項(factors)をX_factorsに保存する
        time_windowを超えた場合、古いfactorsを削除し、最新のfactorsを一番下に保存する
        """
        factors = self.calc_factors(data, i_stock)
        if self.time_window is None:
            self.X_factors[i_stock] = np.vstack(
                [self.X_factors[i_stock], factors])
        elif len(self.X_factors[i_stock]) < self.time_window:
            self.X_factors[i_stock] = np.vstack(
                [self.X_factors[i_stock], factors])
        else:
            #
            self.X_factors[i_stock] = np.roll(
                self.X_factors[i_stock], -1, axis=0)
            self.X_factors[i_stock][-1] = factors
        return None

    def learn(self, y, i_stock):
        epsilon = 0.0001
        X = self.X_factors[i_stock]
        if self._check_rank_deficient(i_stock):
            self.w[i_stock] = np.zeros(len(self.w[i_stock]))
        else:
            self.w[i_stock] = np.linalg.inv(
                X.T.dot(X) + epsilon).dot(X.T).dot(y)

    def _check_rank_deficient(self, i_stock):
        """ ランク落ちを確認
        """
        X = self.X_factors[i_stock]
        return np.linalg.matrix_rank(X.T.dot(X)) < self.num_factors[i_stock]

    def predict(self):  # test dataに対するprediction, これに基づいてモデルの評価を行う．
        """ 最新のfactorsを使用して次の時刻の予測を行う
        """
        y_pred = np.zeros(self.num_stock)
        for i_stock in range(self.num_stock):
            y_pred[i_stock] = self.X_factors[i_stock][-1].dot(self.w[i_stock])
        return y_pred

    def predict_for_X_factors(self):
        """ len(X_factors) < time_windowの時エラー
        """
        y_preds = np.zeros((self.num_stock, self.time_window))
        for i_stock in range(self.num_stock):
            y_preds[i_stock] = self.X_factors[i_stock].dot(self.w[i_stock])
        return y_preds

    def calc_cumulative_error(self, y_true):
        y_preds = self.predict_for_X_factors()
        errors = (y_preds - y_true)**2.0
        errors = np.sqrt(errors.mean(1))
        for i_stock in range(self.num_stock):
            self.cumulative_error[i_stock] = errors[i_stock]
        return self.cumulative_error


4